

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

import matplotlib.pyplot as plt
%matplotlib inline

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
# logistics 
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  beta_regul = tf.placeholder(tf.float32)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + beta_regul*tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_regul : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 18.822466
Minibatch accuracy: 8.6%
Validation accuracy: 10.6%
Minibatch loss at step 500: 2.539135
Minibatch accuracy: 78.1%
Validation accuracy: 76.4%
Minibatch loss at step 1000: 1.955121
Minibatch accuracy: 77.3%
Validation accuracy: 78.4%
Minibatch loss at step 1500: 1.037530
Minibatch accuracy: 82.8%
Validation accuracy: 79.9%
Minibatch loss at step 2000: 0.784823
Minibatch accuracy: 90.6%
Validation accuracy: 80.8%
Minibatch loss at step 2500: 0.867369
Minibatch accuracy: 78.1%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 0.800114
Minibatch accuracy: 82.0%
Validation accuracy: 81.8%
Test accuracy: 89.1%


In [22]:
# exploring the regularzation value
num_steps = 3001
regul_val = [pow(10, i) for i in np.arange(-4, -2, 0.1)]
accuracy_val = []

for regul in regul_val:
  with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
      offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
      batch_data = train_dataset[offset:(offset + batch_size), :]
      batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_regul : regul}
      _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
    accuracy_val.append(accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.glo

In [3]:

plt.semilogx(regul_val, accuracy_val)
plt.grid(True)
plt.title('Test accuracy by regularization (logistic)')
plt.show()

NameError: name 'regul_val' is not defined

In [5]:
# one-layer nn
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  beta_regul = tf.placeholder(tf.float32)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  logits_nn = tf.matmul(lay1_train, weights2) + biases2
  loss_nn = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_nn, tf_train_labels)+ beta_regul*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)))
      
  
  # Optimizer.
  optimizer_nn = tf.train.GradientDescentOptimizer(0.5).minimize(loss_nn)
  
  # Predictions for the training, validation, and test data.
  train_prediction_nn = tf.nn.softmax(logits_nn)
  lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction_nn = tf.nn.softmax(tf.matmul(lay1_valid, weights2) + biases2)
  lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction_nn = tf.nn.softmax(tf.matmul(lay1_test, weights2) + biases2)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_regul : 1e-3}
    _, l, predictions = session.run(
      [optimizer_nn, loss_nn, train_prediction_nn], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction_nn.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction_nn.eval(), test_labels))

Initialized
Minibatch loss at step 0: 672.970581
Minibatch accuracy: 14.1%
Validation accuracy: 25.8%
Minibatch loss at step 500: 200.526489
Minibatch accuracy: 78.9%
Validation accuracy: 77.4%
Minibatch loss at step 1000: 115.143669
Minibatch accuracy: 79.7%
Validation accuracy: 81.1%
Minibatch loss at step 1500: 68.428711
Minibatch accuracy: 87.5%
Validation accuracy: 83.4%
Minibatch loss at step 2000: 41.265156
Minibatch accuracy: 89.1%
Validation accuracy: 84.3%
Minibatch loss at step 2500: 25.153934
Minibatch accuracy: 85.2%
Validation accuracy: 85.3%
Minibatch loss at step 3000: 15.443294
Minibatch accuracy: 88.3%
Validation accuracy: 86.5%
Test accuracy: 93.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [36]:
# one-layer nn
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  beta_regul = tf.placeholder(tf.float32)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  logits_nn = tf.matmul(lay1_train, weights2) + biases2
  loss_nn = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_nn, tf_train_labels))
      
  
  # Optimizer.
  optimizer_nn = tf.train.GradientDescentOptimizer(0.5).minimize(loss_nn)
  
  # Predictions for the training, validation, and test data.
  train_prediction_nn = tf.nn.softmax(logits_nn)
  lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction_nn = tf.nn.softmax(tf.matmul(lay1_valid, weights2) + biases2)
  lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction_nn = tf.nn.softmax(tf.matmul(lay1_test, weights2) + biases2)

In [37]:
num_steps = 100
num = 3

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = step%num
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer_nn, loss_nn, train_prediction_nn], feed_dict=feed_dict)
    if (step % 10 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction_nn.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction_nn.eval(), test_labels))

Initialized
Minibatch loss at step 0: 250.625916
Minibatch accuracy: 12.5%
Validation accuracy: 33.7%
Minibatch loss at step 10: 16.621103
Minibatch accuracy: 96.9%
Validation accuracy: 61.0%
Minibatch loss at step 20: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 60.5%
Minibatch loss at step 30: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 60.5%
Minibatch loss at step 40: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 60.5%
Minibatch loss at step 50: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 60.5%
Minibatch loss at step 60: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 60.5%
Minibatch loss at step 70: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 60.5%
Minibatch loss at step 80: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 60.5%
Minibatch loss at step 90: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 60.5%
Test accuracy: 66.3%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [39]:
# one-layer nn
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  #beta_regul = tf.placeholder(tf.float32)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)  
  logits_nn = tf.matmul(tf.nn.dropout(lay1_train, 0.5), weights2) + biases2
  loss_nn = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_nn, tf_train_labels))
      
  
  # Optimizer.
  optimizer_nn = tf.train.GradientDescentOptimizer(0.5).minimize(loss_nn)
  
  # Predictions for the training, validation, and test data.
  train_prediction_nn = tf.nn.softmax(logits_nn)
  lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction_nn = tf.nn.softmax(tf.matmul(lay1_valid, weights2) + biases2)
  lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction_nn = tf.nn.softmax(tf.matmul(lay1_test, weights2) + biases2)

In [40]:
num_steps = 100
num = 3

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = step%num
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer_nn, loss_nn, train_prediction_nn], feed_dict=feed_dict)
    if (step % 10 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction_nn.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction_nn.eval(), test_labels))

Initialized
Minibatch loss at step 0: 415.223206
Minibatch accuracy: 12.5%
Validation accuracy: 23.5%
Minibatch loss at step 10: 10.771956
Minibatch accuracy: 96.1%
Validation accuracy: 64.4%
Minibatch loss at step 20: 5.219406
Minibatch accuracy: 99.2%
Validation accuracy: 65.3%
Minibatch loss at step 30: 3.143927
Minibatch accuracy: 97.7%
Validation accuracy: 65.5%
Minibatch loss at step 40: 1.865647
Minibatch accuracy: 98.4%
Validation accuracy: 65.0%
Minibatch loss at step 50: 0.080121
Minibatch accuracy: 99.2%
Validation accuracy: 65.6%
Minibatch loss at step 60: 1.234283
Minibatch accuracy: 98.4%
Validation accuracy: 66.2%
Minibatch loss at step 70: 0.910618
Minibatch accuracy: 98.4%
Validation accuracy: 67.0%
Minibatch loss at step 80: 0.935995
Minibatch accuracy: 99.2%
Validation accuracy: 68.4%
Minibatch loss at step 90: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 69.0%
Test accuracy: 75.1%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [47]:
# one-layer nn with learning_rate decay and dropout
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  beta_regul = tf.placeholder(tf.float32)
  #global_step = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  global_step = tf.Variable(0)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  y1 = tf.nn.dropout(lay1_train, 0.5)  
  logits_nn = tf.matmul(y1, weights2) + biases2
  loss_nn = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_nn, tf_train_labels)+ beta_regul*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)))
      
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
  optimizer_nn = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_nn,global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction_nn = tf.nn.softmax(logits_nn)
  lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction_nn = tf.nn.softmax(tf.matmul(lay1_valid, weights2) + biases2)
  lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction_nn = tf.nn.softmax(tf.matmul(lay1_test, weights2) + biases2)

In [48]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_regul : 1e-3}
    _, l, predictions = session.run(
      [optimizer_nn, loss_nn, train_prediction_nn], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction_nn.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction_nn.eval(), test_labels))

Initialized
Minibatch loss at step 0: 778.753296
Minibatch accuracy: 15.6%
Validation accuracy: 27.6%
Minibatch loss at step 500: 214.394272
Minibatch accuracy: 76.6%
Validation accuracy: 79.1%
Minibatch loss at step 1000: 119.866333
Minibatch accuracy: 71.1%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 69.458923
Minibatch accuracy: 80.5%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 41.600876
Minibatch accuracy: 86.7%
Validation accuracy: 83.8%
Minibatch loss at step 2500: 25.421917
Minibatch accuracy: 76.6%
Validation accuracy: 84.1%
Minibatch loss at step 3000: 15.531658
Minibatch accuracy: 82.0%
Validation accuracy: 84.9%
Test accuracy: 92.0%


In [ ]:
# 3-layer nn without dropout: Test accuracy: 94.1%
# 3-layer nn without dropout:Test accuracy: 93.7%
# beta_regul: 0.001438

In [14]:
# multi-layers nn with learning_rate decay and dropout
batch_size = 128
hidden1_nodes = 1024
hidden2_nodes = 512
hidden3_nodes = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  beta_regul = tf.placeholder(tf.float32)
  global_step = tf.Variable(0)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden1_nodes], stddev=np.sqrt(2.0 / (image_size * image_size))))
  biases1 = tf.Variable(tf.zeros([hidden1_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden1_nodes, hidden2_nodes], stddev=np.sqrt(2.0 / hidden1_nodes)))
  biases2 = tf.Variable(tf.zeros([hidden2_nodes]))
  weights3 = tf.Variable(
    tf.truncated_normal([hidden2_nodes, hidden3_nodes], stddev=np.sqrt(2.0 / hidden2_nodes)))
  biases3 = tf.Variable(tf.zeros([hidden3_nodes]))
  weights4 = tf.Variable(
    tf.truncated_normal([hidden3_nodes, num_labels], stddev=np.sqrt(2.0 / hidden3_nodes)))
  biases4 = tf.Variable(tf.zeros([num_labels]))  
  
  # Training computation.
  y1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  y1 = tf.nn.dropout(y1, 0.5)
  y2 = tf.nn.relu(tf.matmul(y1, weights2) + biases2)
  y2 = tf.nn.dropout(y2, 0.5)
  y3 = tf.nn.relu(tf.matmul(y2, weights3) + biases3)
  y3 = tf.nn.dropout(y3, 0.5)
  y4 = tf.matmul(y3, weights4) + biases4
  loss_nn = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(y4, tf_train_labels)+ beta_regul*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + 
                                                                              tf.nn.l2_loss(weights3) +tf.nn.l2_loss(weights4)))
      
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
  optimizer_nn = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_nn,global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction_nn = tf.nn.softmax(y4)

  #validation prediction
  v1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  v2 = tf.nn.relu(tf.matmul(v1, weights2) + biases2)
  v3 = tf.nn.relu(tf.matmul(v2, weights3) + biases3)
  v4 = tf.matmul(v3, weights4) + biases4
  valid_prediction_nn = tf.nn.softmax(v4)

  #test prediction
  t1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  t2 = tf.nn.relu(tf.matmul(t1, weights2) + biases2)
  t3 = tf.nn.relu(tf.matmul(t2, weights3) + biases3)
  t4 = tf.matmul(t3, weights4) + biases4
  test_prediction_nn = tf.nn.softmax(t4)

In [15]:

num_steps = 12001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_regul : 1e-3}
    _, l, predictions = session.run(
      [optimizer_nn, loss_nn, train_prediction_nn], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction_nn.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction_nn.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6.379360
Minibatch accuracy: 9.4%
Validation accuracy: 10.9%
Minibatch loss at step 500: 2.400925
Minibatch accuracy: 85.9%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 1.607831
Minibatch accuracy: 82.8%
Validation accuracy: 84.8%
Minibatch loss at step 1500: 1.066145
Minibatch accuracy: 87.5%
Validation accuracy: 85.7%
Minibatch loss at step 2000: 0.781253
Minibatch accuracy: 90.6%
Validation accuracy: 86.0%
Minibatch loss at step 2500: 0.835401
Minibatch accuracy: 85.9%
Validation accuracy: 85.7%
Minibatch loss at step 3000: 0.735455
Minibatch accuracy: 84.4%
Validation accuracy: 86.2%
Minibatch loss at step 3500: 0.701375
Minibatch accuracy: 85.2%
Validation accuracy: 86.1%
Minibatch loss at step 4000: 0.640709
Minibatch accuracy: 89.1%
Validation accuracy: 86.5%
Minibatch loss at step 4500: 0.636300
Minibatch accuracy: 86.7%
Validation accuracy: 86.3%
Minibatch loss at step 5000: 0.639349
Minibatch accuracy: 87.5%
Validation accuracy